<a href="https://colab.research.google.com/github/dgsmith1988/AEC-Challenge/blob/main/framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ECSE-552 HW4
## Weather multivariate forecast

### Initial configurations

In [9]:
# Autoreload : automatically reload imported scripts so that changes on them take effect.
%load_ext autoreload
%autoreload 2

In [1]:
# Install required libraries. 
# /dev/null hides the output except for errors
# NOTE: if an error is thrown, then please restart Runtime and run again!
!pip install pytorch-lightning

     |████████████████████████████████| 582 kB 4.2 MB/s 
     |████████████████████████████████| 136 kB 71.3 MB/s 
     |████████████████████████████████| 596 kB 56.4 MB/s 
     |████████████████████████████████| 398 kB 66.0 MB/s 
     |████████████████████████████████| 1.1 MB 54.5 MB/s 
     |████████████████████████████████| 271 kB 75.6 MB/s 
     |████████████████████████████████| 144 kB 77.2 MB/s 
     |████████████████████████████████| 94 kB 4.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow

### Configure directories/paths

In [1]:
import os
mount_point = '/content/drive'
from google.colab import drive
drive.mount(mount_point, force_remount= True)

Mounted at /content/drive


In [2]:
user = "rgmartin"
repo = "ECSE552-HW4"
branch = "graham"
src_dir = "data"
datafiles = ["weather_test.csv", "weather_train.csv", "weather_test_debug.csv", "weather_train_debug.csv"]
pyfiles = ['train.py', 'models.py', 'feature_extraction.py', 'dict_logger.py']
folders = ['data/']*len(datafiles) + ['code/']*len(pyfiles)

for src_folder, file in zip(folders, datafiles + pyfiles):
    !rm {file}
    url = f"https://github.com/{user}/{repo}/raw/{branch}/{src_folder}/{file}"
    print(url)
    !wget {url} > /dev/null
  

https://github.com/rgmartin/ECSE552-HW4/raw/graham/data//weather_test.csv
--2022-04-13 00:19:47--  https://github.com/rgmartin/ECSE552-HW4/raw/graham/data//weather_test.csv
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rgmartin/ECSE552-HW4/graham/data/weather_test.csv [following]
--2022-04-13 00:19:48--  https://raw.githubusercontent.com/rgmartin/ECSE552-HW4/graham/data/weather_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1404272 (1.3M) [text/plain]
Saving to: ‘weather_test.csv’

weather_test.csv    100%[===================>]   1.34M  --.-KB/s    in 0.07s   

2022-04-13 00:19:48 (2

### Run/Train the models

In [ ]:
from models import DenseNetwork
from feature_extraction import WeatherTimeSeriesDataset
from train import *

test_csv_path = "weather_test.csv"
train_csv_path = "weather_train.csv"
# test_csv_path = "weather_test_debug.csv"
# train_csv_path = "weather_train_debug.csv"

case = "Validate"

k = 4

output_features = ['p (mbar)', 'T (degC)', 'rh (%)', 'wv (m/s)']
input_features = output_features + ['Date Time', 'Tpot (K)', 'VPmax (mbar)', 'rho (g/m**3)', 'max. wv (m/s)']

max_epoch = 10
batch_size = 128

num_input = k * len(input_features)
num_output = len(output_features)

model = DenseNetwork([num_input, 30, 15, num_output], torch.nn.ReLU(), torch.nn.functional.mse_loss, p=None)
# model = DenseNetwork([num_input, 30, 15, num_output], torch.nn.ReLU(), torch.nn.functional.mse_loss, p=.15)
# model = DenseNetwork([num_input, 30, 15, num_output], torch.nn.ReLU(), torch.nn.functional.mse_loss, p=.30)
# model = DenseNetwork([num_input, 30, 15, num_output], torch.nn.ReLU(), torch.nn.functional.mse_loss, p=.45)


train_dataset = WeatherTimeSeriesDataset(train_csv_path, input_features, output_features, k)
if case == "Validate":
    train_dataset, val_dataset = split_dataset(train_dataset, .8)
    train_validate_model(model, train_dataset, val_dataset, max_epoch=max_epoch, batch_size=batch_size)
elif case == "Test":
    test_dataset = WeatherTimeSeriesDataset(test_csv_path, input_features, output_features, k)
    train_test_model(model, train_dataset, test_dataset, max_epoch=max_epoch, batch_size=batch_size)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /content/DictLogger/0.1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type       | Params
-------------------------------------
0 | model | Sequential | 1.6 K 
-------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.007     Total estimated model params size (MB)


Creating measurements path...
Running on Colab
Initializing trainer...


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]